In [73]:
import torch
from torch import nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim

FEAT_DIM = 38
INTER_DIM = 20
RES_DIM = 2
LR = 0.01
NB_EPOCH = 1
TEST_SPLIT = 20
VALID_SPLIT = 10

criterion = nn.MSELoss()

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fcTop = nn.Linear(in_features=FEAT_DIM, out_features=INTER_DIM) 
        self.fcMid = nn.Linear(in_features=INTER_DIM, out_features=INTER_DIM)
        self.fcBot = nn.Linear(in_features=INTER_DIM, out_features=RES_DIM)
        
    def forward(self, x):
        x = F.relu(self.fcTop(x))
        for i in range(2):
            x = F.relu(self.fcMid(x))
        x = self.fcBot(x)
        return x

def testTrainValidSplit(dataset, test_split=20, valid_split=10):
    
    def split(dt, percen):
        total_len = dt.size()[0]
        first_len = total_len*percen//100
        second_len = total_len-first_len
        second_idx = np.random.choice(total_len, second_len, replace=False)
        mask = np.ones(total_len)
        mask[second_idx] = 0
    
        second_set = dt[second_idx, :]
        first_set = dt[np.squeeze(np.argwhere(mask)),:]
        return first_set, second_set
    
    test_set, train_set = split(dataset, test_split)
    valid_set, train_set = split(train_set, valid_split)
    
    return test_set, train_set, valid_set

In [76]:
#------------------------Dataset preparation-------------------------
#Loading
df = pd.read_csv("league-of-legends-diamond-ranked-games-10-min/high_diamond_ranked_10min.csv")
df = df.drop(columns=['gameId'])
dataset = torch.tensor(df.values)

#Random train/test split
test_set, train_set, valid_set = testTrainValidSplit(dataset, test_split=TEST_SPLIT, valid_split=VALID_SPLIT)
train_target = train_set[:,0]
train_data = train_set[:,1:]
test_target = test_set[:,0]
test_data = test_set[:,1:]
valid_target = valid_set[:,0]
valid_data = valid_set[:,1:]


In [80]:
#------------------------Model preparation-------------------------
net = Net()
optimizer = optim.SGD(net.parameters(), lr=LR)

"""
for e in range(NB_EPOCH):
    optimizer.zero_grad()
    output = net(train_set)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
"""


'\nfor e in range(NB_EPOCH):\n    optimizer.zero_grad()\n    output = net(train_set)\n    loss = criterion(output, target)\n    loss.backward()\n    optimizer.step()\n'